#### Point Map Visualization

Point map visualization is natively supported, if your table outputs longitude-latitude pairs. It is recommended to limit the query to not too many rows for performance reasons.

> [!NOTE]
> We are using Spatial Functions below, and as of 2025-07-16, you need to use Databricks Runtime 17.1+ for this. It is expected that this feature becomes available on Serverless Compute and DBSQL as well soon, see [here](https://www.linkedin.com/feed/update/urn:li:activity:7351276934338641920/).

In [0]:
%pip install duckdb lonboard shapely --quiet

In [0]:
import duckdb
from lonboard import viz

HTTP_BLOCKED = True

if HTTP_BLOCKED:
    import os
    from urllib.parse import urlparse

    import requests

    ARCHITECTURE = "linux_amd64"
    duckdb_version = duckdb.__version__
    url = f"https://extensions.duckdb.org/v{duckdb_version}/{ARCHITECTURE}/httpfs.duckdb_extension.gz"

    output_file = os.path.basename(urlparse(url).path)
    response = requests.get(url, timeout=30)
    response.raise_for_status()
    with open(output_file, "wb") as f:
        f.write(response.content)

    duckdb.install_extension(output_file)

    os.remove(output_file)

    duckdb.sql("SET custom_extension_repository='https://extensions.duckdb.org'")

duckdb.sql("install spatial; load spatial")

In [0]:
spark.read.parquet("s3://overturemaps-us-west-2/release/2025-06-25.0/theme=places/type=place").createOrReplaceTempView("place")

In [0]:
%sql
with t as (
  select
    *,
    st_geomfromwkb(geometry, 4326) geom,
    st_x(geom) lon,
    st_y(geom) lat
  from
    place
)
select
  *
from
  t
where
  bbox.xmin between 4.72 and 5.08
  and bbox.ymin between 52.27 and 52.44
  and categories.primary = "restaurant"
limit 10000

Databricks visualization. Run in Databricks to view.

In [0]:
dfa = _sqldf.toArrow()

In [0]:
viz(duckdb.sql("select st_geomfromwkb(geom.wkb) geom from dfa")).as_html()